In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.Client):
    for _ in range(100):
        loss = client.send_grad_get_loss()
        # print(f"Client {i}: {loss}")
        client.apply_global_step()
        
def run_master(master: mfl.algorithms.gd.Master):
    for i in range(100):
        print(f"Master: {master.round()}")

In [3]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()

eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], len(eval_data[0]) // NUM_CLIENTS, dim=0), torch.split(eval_data[1], len(eval_data[1]) // NUM_CLIENTS, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.gd.get_master_and_clients(
    lr=200,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
)

In [4]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Master: 0.6610914468765259
Master: 0.6171865463256836
Master: 0.590644896030426
Master: 0.5646098256111145
Master: 0.544524610042572
Master: 0.5266827940940857
Master: 0.5115221738815308
Master: 0.49819308519363403
Master: 0.48645859956741333
Master: 0.47601619362831116
Master: 0.46666380763053894
Master: 0.45823439955711365
Master: 0.45059579610824585
Master: 0.4436403810977936
Master: 0.4372800290584564
Master: 0.4314415156841278
Master: 0.4260633885860443
Master: 0.4210938513278961
Master: 0.41648876667022705
Master: 0.4122101664543152
Master: 0.4082253873348236
Master: 0.4045061469078064
Master: 0.4010276198387146
Master: 0.3977680504322052
Master: 0.3947083353996277
Master: 0.39183151721954346
Master: 0.3891226053237915
Master: 0.3865681290626526
Master: 0.384156197309494
Master: 0.38187599182128906
Master: 0.37971779704093933
Master: 0.37767288088798523
Master: 0.3757333755493164
Master: 0.3738919794559479
Master: 0.37214231491088867
Master: 0.3704782724380493
Master: 0.368894398

In [5]:
clients[0].data_sender.n_bits_passed

217600